In [2]:
import openai
import os
#from langchain.llms import CTransformers

In [3]:
#Loading the model
'''
def load_model():
    # Load the locally downloaded model here
    llm = CTransformers(
        model = "/home/txjia/work/freelance_demo/models/codellama-7b-instruct.ggmlv3.Q4_0.bin",
        model_type="llama",
        max_new_tokens = 1096,
        temperature = 0.2,
        repetition_penalty = 1.13
        #stream=True
        #gpu_layers = 10
    )

    return llm
'''

'\ndef load_model():\n    # Load the locally downloaded model here\n    llm = CTransformers(\n        model = "/home/txjia/work/freelance_demo/models/codellama-7b-instruct.ggmlv3.Q4_0.bin",\n        model_type="llama",\n        max_new_tokens = 1096,\n        temperature = 0.2,\n        repetition_penalty = 1.13\n        #stream=True\n        #gpu_layers = 10\n    )\n\n    return llm\n'

In [4]:
def get_completion(prompt, model="gpt-3.5-turbo"):
#def get_completion(prompt, model=load_model()):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

In [5]:
text = f"""
       <Context:Any,
    Phrase: It would be good to have free shipping when I spend more than $50.>
"""


prompt = f"""
This is a context based sentiment analysis task. Here is the tutorial for it:
<tutorial>
Read the phrase from a customer's feedback and the context associated with the phrase to decide the sentiment category for the phrase based on the context. 
The meaning of "Phrase" ande "Context" sentiment judgments are below: 
-Phrase: A free-form customer comment in the survey. 
-Context: The question asked in the survey which is the context associated with the answered comments from the customer. they are classified into four: “positive”, “neutral”, “suggestion”, and “negative”. 


Below we will show you some questions (used as context) and possible answers(used as phrase) from different customers. 

Question example 1: What do you like about your experience? 
Answer 1: Service is good
Answer 2: Speed of service
Answer 3: Nothing, the overall service is not good.
Answer 4: The hotel is far from the street.

The sentiment for each phrase:
Answer 1: “Service is good”  is positive
Answer 2: “Service” is positive because it infers that the customer likes the service.
Answer 3: “Nothing, the overall service is not good” is negative. \
    The customer might not agree with the question being prompted for the positive feedback and they can write anything. \
    In this case, they want to express their opinions that service is not good. 
Answer 4: “The hotel is far from the street” is a statement in the neutral context. \
    In a positive context, it implies that this statement is a positive.

Tip:
    -If the statement in a positive context sounds like the reason (“the hotel is far from the street), then it is positive as it implies the customer likes the reason as mentioned.
    -If the statement in a positive context does not sound like the reason, then it is no clear opinion.

Question example 2: What do you not like about your experience? 
Answer 1: Service is not good
Answer 2: Speed of service
Answer 3: Overall service is good and I have nothing to complain about.
Answer 4: The hotel is far from the street

The sentiment for each phrase:
Answer 1: “Service is not good” is negative
Answer 2: “Service” is negative because it infers that the customer did not like the service.
Answer 3: “Overall service is good and I have nothing to complain about” is positive. The customer might not agree with the question being prompted for the negative feedback and they can definitely write anything. In this case, they want to express their opinions that the service is good. 
Answer 4: “The hotel is far from the street” is a statement. In a negative context, it implies that this statement is a negative sentiment. (Explanation: some guests might not like the hotel being far away from the street as it’s inconvenient to go to places)

Question example 3: Please tell us why you gave the score of 10. 
<Explanation: The customer gave a score of 10 in “likelihood to recommend” question.  The follow-up question asks the reason for the rating.>

Likelihood to recommend question: “How likely are you to recommend the product or service?”. Also the context can be said like please tell us why you gave the score of 10(highest high).
Answer 1: I gave a 10 because of the service. 
Answer 2: Speed of service

The answer 1 and answer 2 are all positive

Question example 4: Please tell us why you gave the score of 1(lowest score). 
Answer 1: I gave a 1 because of the service.
Answer 2: Speed of service

The answer 1 and answer 2 are all negative

“The service” can be positive or negative depending on the score (high or low).

Question example 5: What can we do to improve your experience? 
Answer 1: I am not happy with the service.
Answer 2: Better service/Quicker service
Answer 3: Speed of service
Answer 4: Overall service is good and I have nothing to complain about.

The sentiment for each phrase:
Answer 1: “I am not happy with the service.” is negative
Answer 2: “Better Service” is a suggestion. The customer suggests the company provide better service.
Answer 3: “Service” is a suggestion based on the context. The customer suggests the company improve the service.
Answer 4: “Overall service is good and I have nothing to complain about” is positive. The customer might not agree with the question being prompted for the suggestion feedback. In this case, they want to express their opinions that the service is good. 


Here are four variants of context: positive, neutral, negative, and suggestion. Here are some examples of them:
1. Positive context
 - What do you like about your experience?
 - What is the reason for the score of x? (x can be any high score)

2. Neutral context
 - What is your overall experience?
 - Is there anything else you would like to ask or tell us about? 

3. Negative context
 - What do you NOT like about your experience?
 - What is the reason for the score of x? (x can be any low score)

4. Suggestion context
 - What can we do to improve your experience?
 - What would you suggest to make your experience better?

Here are the sentiment labels of the phrase on the condition of context: 
    - No Clear Opinion (Statement of Fact/Description or Neutral Sentiment)
    - Positive (The phrase contains positive sentiment)
    - Strongly Positive (The phrase contains strongly positive sentiment)
    - Negative (The phrase contains negative sentiment)
    - Strongly Negative (The phrase contains strongly negative sentiment)
    - Suggestion  (The phrase makes suggestions to the company to take action)
    - Mixed Opinion (The phrase contains both positive and negative sentiment)
    - Incorrect Language or Gibberish (The phrase is not in the right language or it’s completely gibberish)
    - Unsure: the text is clear. If the text is not clear because it’s not a complete sentence or it might require more information, choose “No Clear Opinion”

Here are the examples of labelling the phrase on the condition of context.
1. No Clear Opinion
    In all the sub-class sutiation, all phrase are labelled as No Clear Opinion
    1) Phrase is purely an observation, description, or a statement of fact;  
        <Context: any, Phrase: This was our anniversary weekend get-away.>        
        Sentiment: No clear opinion

    2) the sentiment is neutral about the product or company.
        <Context: Any, Phrase: The product is okay/fine.>        
        Sentiment: No clear opinion       


    3) If the phrase is not clear but mostly descriptive, or you might need more context (surrounding texts in the comment) to make the judgment, label “No clear opinion”. This usually results from poor segmentation, too many typos and the text itself does not imply any sentiment.
        <Context: Any, Phrase: I say all that to say this.>        
        Sentiment: No clear opinion
    
    4) a question with a neutral tone that don not imply any positive or negative sentiment.
        <Context: Any, Phrase: When will the product be released?>        
        Sentiment: No clear opinion

    
    5) The phrase describes the product is not lacking in quality or customers are not experiencing any issues or their expectations are met, \
        sometimes phrases with clear sentiments or in positive context and note the sentiment changes
        <Context: Neutral,  Phrase: The cloth is not see-through!>      
        Sentiment: No clear opinion    
    
    Tips:

        - Adjectives like soft/hard, thin/thick, or cold/hot do not exhibit inherent sentiment and should be labeled neutral or no clear opinion as it can be either positive or negative depending on personal preferences. 
        - Adjectives like good, nice, or awesome, which are inherently positive should be labeled positive sentiment.
        - Adjectives like bad, terrible, or lousy, which are inherently negative should be labeled negative sentiment.


    6) The phrase only contains nouns in neutral context. The noun does not contain any positive/negative sentiment.  They become positive or negative only in positive or negative contexts but remain no clear opinion in the neutral context.
        <Context: Neutral,   Phrase: Professionalism.>      
        Sentiment: No clear opinion
    

    7) The sign-off language such as “thank you”, “thanks”
        <Context: Neutral,  Phrase: Thank you REDACT_NAME>        
        Sentiment: No clear opinion

    
2. Positive:
Phrase contains any positive opinion from the customer regarding customer experience. Adjectives that are always associated with good customer experiences (“comfortable bed”, “fast/quick service”, “friendly/knowledgeable/helpful staff”, “inexpensive product/low cost”) in any context should be labeled positive sentiment.
    <Context: Any, Phrase: You delivered on time.>
    Sentiment: Positive  


3. Strongly Positive
    Phrase generally contains a strong tone for positive sentiment. The customer might use the modifier like "very", "extremely", "really" or any strong adjective like great, awesome, or best. 
    <Context:Any,  Phrase: Jimmy at the Craps table was AWESOME and a great teacher for this first-timer!>
    Sentiment: Strongly positive
    
4. Negative
    Phrase contains 
        1) any negative opinion from the customer regarding customer experience (anger, frustration, dissatisfied) or 
        2) any issues the customer is experiencing
    Adjectives that are always associated with poor customer experiences (“uncomfortable bed”, “slow/poor service”, “rude staff”, “expensive product/high cost”, “unacceptable responses”) should be labeled negative sentiment.

        <Context:Any,  Phrase: She did not seem to understand the issue or want to.>
        Sentiment: Negative
        
5. Strongly Negative
    Phrase generally contains a strong tone for negative sentiment. The customer might use the modifier like "very", "extremely", "really", "worst" or any strong adjective like worst.
    <Context:Any, Phrase: I was very disappointed as I had been anticipating enjoying this buffet for a long time.>
    Sentiment: Strongly negative


6. Mixed Opinion
    Phrase contains both positive and negative sentiments. 
    <Context:Any, Phrase: Even though the restaurant choices were limited, we enjoyed Charlie’s food so much we ate there twice in one day.>
    Sentiment: Mixed opinion   

7 Suggestion
    Phrase contains some specific actions or suggestions that lead to fulfilling the customer's wish/intent. 

     - In any context, some common suggestions have patterns "It would be nice/great/good ...", "you need to", "You have to", "you should", "I wish", "..would be helpful", "...would help.."
     - In a negative or suggestion context, phrases starting with comparative words (better, lower, fewer, easier, etc) imply the customers want the company to take action to make certain things better, lower, fewer, or easier, and hence it’s a suggestion.
     - In a suggestion context, phrases with “Being able to do something” or “ability to do something” imply the customers want the company to fulfill their needs, and hence it’s a suggestion.
     - The phrase contains “I would like to” but it’s not a suggestion to the company to take action to fulfill the customers’ needs.
        ◦ I would like to comment that the service is slow
        ◦ I wish I can give you a 10 but the service this time is slow
     - The phrases are descriptions of the problem or complaints. It does not contain any suggested “action” to take.
        ◦ The service is slow
        ◦ The price is too high

    <Context:Any,   Phrase: Provide the Spanish mobile app>
    Sentiment: Suggestion  

    
8. Incorrect language or gibberish
If the text is not in the correct language or the phrase does not make any sense to you (gibberish), then choose "incorrect language or gibberish". If it’s just the part of the sentence is gibberish but you still can interpret the meaning, choose your best sentiment judgment. 
Note that “%d” is the symbol to replace sensitive numeric data (see Step above) as we remove personal sensitive information from the data. You should interpret them as some numbers instead of gibberish.
    
    <Context:Any,  Phrase: jjo>
    Sentiment: gibberish    

9. Unsure
    The text is clear to you but you don’t know how to annotate it, that is the probability of being divided into any category is almost equal

Above is all about the tutorial,
</tutorial>

Please do the context based sentiment analysis:
    - Read the phrase and context. Due to data privacy and security reasons, we replace the company name with REDACT_COMPANY, emails with REDACT_EMAIL and digit with '#', so you might see repeating '###' in the comment. 
    - The sentiment category should one of No Clear Opinion,positive, strongly Positive, negative, strongly Negative, suggestion, mixed Opinion, incorrect Language or Gibberish,and Unsure.

The context and phrase: ```{text}```
"""


In [6]:
response = get_completion(prompt)
print(response)

The sentiment category for the given phrase and context is "Suggestion".
